# Quickstart tutorial: Enter the _Vivarium_

First follow the installations instruction in the README of the repository. If everything works correctly, you should have launched the web interface.

## Introduction to the web interface

The web interface has two main components:
- The simulator plot on the left (in blue)
- The configurations on the right (in red)

![e](img/web_interface.png "Interface")

### Simulator plot

The plot displays the current state of the simulator, including its agents and objects.

From top to bottom can be found:

- **The start/stop button for the server**, which manages the simulation on the server side
- **The start/stop button for the plot update**, which sets if the plot should periodically refresh or not; the timestep for the refresh can be set with the slider next to it
- **The simulator plot**, with the agents and objects; on the right are some tools that can be used to select these entities

### Entity configurations

The configs can be used to modify the agents, objects, or simulator properties.
To show one config tab, click on the appropriate **`Show Configs`** toggle.

#### Agents / Objects configs
These two tabs are quite similar in structure.
From top to bottom can be found:
- **The selected list**, which indicates which entities are currently selected, either by selecting them on the plot or directly in this list; <u>the selected entities are the ones that will be affected by the next points</u>
- **The panel configs**, which are used to select what is visible on the plot; checking or unchecking these boxes will make the appropriate element appear / dissapear from the plot
- **The configs**, which manages all the properties of the agents / objects; for more details on what each config does, please refer to the documentation

#### Simulator configs
As there is only one simulator that is always visible, this config tab only shows the simulator properties. Please refer to the documentation for further details on what each config does.

## Controlling the simulator programmatically

Instead of controlling the simulator from the web interface, you can also control it programmatically, e.g. directly from this Jupyter notebook. First make sure that the simulator is stopped in the web interface. Then execute:

In [ ]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

The newly created `controller` object provides all the functionalities to easily interact with the simulator server. Let's look for example at the agents present in the environment:

In [ ]:
controller.agents

This is a list of `Agent` objects, one per agent in the environment. Let's for example access the configuration of the first agent in this list and look at its attribute using the Python `.__dict__` syntax:

In [ ]:
controller.agents[0].__dict__

We see the value of different attributes of the agent, for example its position, diameter or color. Let's make the first agent larger by increasing its `diameter` attribute:

In [ ]:
controller.agents[0].diameter = 10.

If you now look at the web interface, you should see that one of the agent is now twice larger than the others. All changes made in this notebook will be automatically visible in the web interface, and vice versa. One can for example also change the color of an agent, let's do it on the third agent in the list now (i.e. the agent with index 2):

In [ ]:
controller.agents[2].color = 'green'

The same principles apply to objects, which are accessible through `controller.objects`.

### Defining agent's behaviors

We can define the behavior of an agent as a Python function taking as argument an `Agent` and returning the values of the left and right motors, in this order. Within the body of the function, one can access all the agents attribute. Usually, defining a behavior requires to access the value of the agent sensors. This done through the `agent.left_prox` and `agent.right_prox` attributes, that return the value of the left and right sensors, respectively.

Let's define four behaviors, here corresponding to the four canonical behaviors of [Braitenberg Vehicles](https://en.wikipedia.org/wiki/Braitenberg_vehicle):

In [ ]:
def aggression(agent):
    return agent.right_prox, agent.left_prox

def fear(agent):
    return agent.left_prox, agent.right_prox

def love(agent):
    return 1. - agent.left_prox, 1. - agent.right_prox

def shy(agent):
    return 1. - agent.right_prox, 1. - agent.left_prox

The code above only declares the behaviors, now we need to attach them to the agents in the simulator. Let's attach the `shy` behavior to the first five agents and set their color to blue ; and the `aggression` behavior to the four last agents and set their color to red:

In [ ]:
for ag in controller.agents[:5]:
    ag.color = 'blue'
    ag.detach_all_behaviors()
    ag.attach_behavior(shy)
for ag in controller.agents[5:]:
    ag.color = 'red'
    ag.detach_all_behaviors()
    ag.attach_behavior(aggression)

All agents are now equipped with a behavior. We can launch the simulation with the code below ; then stop it with the code on the next line.

In [ ]:
controller.run(threaded=True)

In [ ]:
controller.stop()